## 🧐 Klasyfikacja nowego obrazu

Teraz załadujemy pojedyncze zdjęcie i wprowadzimy do modelu, aby sprawdzić, jaki typ komórki zostanie przewidziany.  

Model zwróci **wektor prawdopodobieństw** dla 4 klas, a my wybierzemy tę z najwyższą wartością.


In [3]:
import tensorflow as ts
import numpy as np
from tensorflow.keras.preprocessing import image

IMG_SIZE = (240, 240)

# Wczytanie modelu
model = ts.keras.models.load_model("blood_cell_classifier.h5")

# Przetwarzanie obrazu do testu
img_path = "images/TEST_SIMPLE/NEUTROPHIL/_0_1966.jpeg"
# img_path = "images/TEST_SIMPLE/MONOCYTE/_0_5020.jpeg"
img = image.load_img(img_path, target_size=IMG_SIZE)
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Predykcja
predictions = model.predict(img_array)
class_names = ["EOSINOPHIL", "LYMPHOCYTE", "MONOCYTE", "NEUTROPHIL"]
predicted_class = class_names[np.argmax(predictions)]
print(f"Model przewiduje: {predicted_class}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
Model przewiduje: NEUTROPHIL
